In [1]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [2]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [3]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [4]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [5]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [6]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [7]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [24]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[5.5926e+02, 1.1000e+00],
         [2.5010e+02, 2.1548e-03],
         [2.9215e+02, 2.5481e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[1.6799e+02, 1.1000e+00],
         [5.0015e+01, 1.4014e-01],
         [5.1023e+01, 1.3013e-01],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[1.6605e+02, 1.1000e+00],
         [4.6033e+01, 7.6923e-02],
         [5.6050e+01, 1.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[3.4215e+02, 1.1000e+00],
         [1.0808e+02, 1.7162e-02],
         [1.1807e+02, 6.4569e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.2623e+02, 1.1000e+00],
         [7.0065e+01, 8.

In [25]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 149, 256, 256, 256, 256, 175, 256, 256, 256, 256, 256])


In [26]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [27]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [28]:
tmp_original['precursor_mz']

tensor([559.2603, 167.9936, 166.0521, 133.0608, 325.1911, 522.3560, 854.5700,
        323.9880, 539.1200, 342.1547, 326.2339, 383.0497])

In [29]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+'],)

In [30]:
tmp_original['mol_freq']

tensor([ 81.,  24.,  16.,  57., 405., 301.,  73.,  35.,  78.,  63.,   3., 200.])

In [31]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [32]:
sum(tmp_original['labels'])

tensor(12)

In [33]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  405,  661,  917, 1173, 1429, 1604, 1860, 2116, 2372, 2628])

In [34]:
len(tmp_original['labels'])

2884

In [35]:
tmp_original['batch_ptr']

tensor([256, 149, 256, 256, 256, 256, 175, 256, 256, 256, 256, 256])

In [36]:
sum(tmp_original['batch_ptr'])

tensor(2884)

In [37]:
len(tmp_original['candidates'])

2884

In [38]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [39]:
len(tmp_original['candidates_smiles'])

2884

# MSnRetrieval

In [40]:
from massspecgym.featurize import SpectrumFeaturizer

In [41]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [42]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [43]:
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.json"

In [44]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 16476
MSnRetrievalDataset length: 16476


In [45]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [46]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 12536
Val dataset size: 1952


In [47]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

{'spec': DataBatch(x=[128, 1039], edge_index=[2, 232], batch=[128], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([1096.6262,  750.3531,  419.0939,  456.1321,  435.1133,  414.1248,
         334.1397,  389.1408,  285.1921,  578.3501,  612.3603,  339.2166]), 'adduct': ['[M+NH4]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H-H2O]+'], 'identifier': ['0009772_0000000', '0094731_0000000', '0043678_0000000', '0048544_0000000', '0039329_0000000', '0021262_0000000', '0069271_0000000', '0070623_0000000', '0085579_0000000', '0053616_0000000', '0007343_0000000', '0006342_0000000'], 'mol_freq': tensor([2., 2., 1., 1., 1., 1., 1., 2., 2., 3., 3., 4.]), 'smiles': ['CC(=CCC[C@@](C)([C@H]1CC[C@@]

In [48]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([140,  85, 256, 256, 197, 177, 256, 256, 256,  55,  36, 256],
       dtype=torch.int32)


In [49]:
def print_key_types(obj, obj_name):
    print(f"Data types in {obj_name}:")
    for key, value in obj.items():
        print(f"  {key}: {type(value)}")

# Print data types for both objects
print_key_types(tmp_original, "tmp_original")
print_key_types(tmp_msn, "tmp_msn")

Data types in tmp_original:
  spec: <class 'torch.Tensor'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  identifier: <class 'list'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>
Data types in tmp_msn:
  spec: <class 'abc.DataBatch'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  identifier: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [34]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [35]:
tmp_msn['spec'] 

DataBatch(x=[139, 1039], edge_index=[2, 254], batch=[139], ptr=[13])

In [36]:
tmp_msn['precursor_mz']

tensor([395.2078, 472.1359, 196.0968, 250.0202, 340.1292, 342.0707, 393.2072,
        774.2968, 586.2177, 454.0486, 392.1969, 350.1266])

In [37]:
tmp_msn['mol_freq']

tensor([1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1.])

In [38]:
tmp_msn['identifier']

['0038826_0000000',
 '0059300_0000000',
 '0098836_0000000',
 '0071484_0000000',
 '0051910_0000000',
 '0031829_0000000',
 '0094508_0000000',
 '0012337_0000000',
 '0060572_0000000',
 '0075294_0000000',
 '0050280_0000000',
 '0059260_0000000']

In [39]:
tmp_msn['adduct']

['[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+']

In [40]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [41]:
tmp_msn['smiles']

['CC1=CC=C(C=C1)N2CCN(C2=O)CC(=O)N3CCC(C3)OC4=NC=C(C=C4)C',
 'CC1=CC=C(C=C1)S(=O)(=O)N2CCCC[C@@H]2C(=O)NC3=NC(=CS3)C4=CC=C(C=C4)OC',
 'C[C@](CC1=CC=C(C=C1)O)(C(=O)O)N',
 'C1CSC(=O)C1NC(=O)CSCC(=O)O',
 'CC1=CC(=NO1)NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)C3=CC=CO3',
 'C1C(=O)N(N=C2N1C3=CC=CC=C3S2)CC(=O)C4=CC=C(C=C4)F',
 'C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@@]4([C@]3([C@H](C[C@@]2([C@]1(C(=O)CO)O)C)O)F)C',
 'C1CC(C=C1)C(=O)OCC2C(C(C(C(O2)OC3=C(C=C(C=C3)O)COC(=O)C4(C(CCC(=O)C4OC(=O)C5CCC=C5)O)O)OC(=O)C6CCC=C6)O)O',
 'CC(C)(C1=CC=CC=C1CC[C@H](C2=CC=CC(=C2)/C=C/C3=NC4=C(C=CC(=C4)Cl)C=C3)SCC5(CC5)CC(=O)O)O',
 'C=CC1=C(N2[C@@H]([C@@H](C2=O)NC(=O)/C(=N/OCC(=O)O)/C3=CSC(=N3)N)SC1)C(=O)O',
 'COCCN1C2=CC=CC=C2C=C1C(=O)NCCC3=CNC4=C3C=C(C=C4)OC',
 'CN1C(=O)COC2=C(C=C(C=C21)Cl)C(=O)NC3CN4CCC3CC4']

In [42]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [43]:
sum(tmp_msn['labels'])

tensor(12)

In [44]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1523, 1779, 1798, 1882, 1918, 2174])

In [45]:
len(tmp_msn['labels'])

2430

In [46]:
tmp_msn['batch_ptr']

tensor([256, 256, 256, 256, 256, 243, 256,  19,  84,  36, 256, 256],
       dtype=torch.int32)

In [47]:
sum(tmp_msn['batch_ptr'])

tensor(2430, dtype=torch.int32)

In [48]:
len(tmp_msn['candidates'])

2430

In [49]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [50]:
tmp_msn.keys() == tmp_original.keys()

True

In [51]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [52]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

False

In [68]:
tmp_original['batch_ptr'].dtype

torch.int64

In [67]:
tmp_msn['batch_ptr'].dtype

torch.int32

In [53]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(2430, dtype=torch.int32), 2430)

In [54]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(2994), 2994)

In [55]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [56]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [57]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [58]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [59]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [60]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [61]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [62]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [63]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [64]:
len(tmp_msn['candidates_smiles'])

2430